In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import pulp

# Read the distance matrix from the Excel file with headers and row names
df = pd.read_excel('TSP.xlsx', index_col=0)

# Extract the city names
city_names = df.columns.tolist()

# Convert the DataFrame to a NumPy array for the distance matrix
distances = df.to_numpy()

n = len(distances)  # Number of cities
cities = list(range(n))

# Create the problem variable to contain the problem data
problem = pulp.LpProblem("TSP", pulp.LpMinimize)

# Create variables: x[i, j] is 1 if the path goes from city i to city j, else 0
x = pulp.LpVariable.dicts("x", (cities, cities), cat='Binary', lowBound=0, upBound=1)

# Objective function: Minimize the sum of distances
problem += pulp.lpSum([distances[i][j] * x[i][j] for i in cities for j in cities if i != j])

# Each city must be arrived at from exactly one other city
for j in cities:
    problem += pulp.lpSum([x[i][j] for i in cities if i != j]) == 1

# Each city must be left by exactly one other city
for i in cities:
    problem += pulp.lpSum([x[i][j] for j in cities if i != j]) == 1

# Subtour elimination using Miller-Tucker-Zemlin (MTZ) constraints
u = pulp.LpVariable.dicts("u", cities, lowBound=0, upBound=n-1, cat='Continuous')
for i in cities:
    for j in cities:
        if i != j and i != 0 and j != 0:
            problem += u[i] - u[j] + n * x[i][j] <= n - 1

# Solve the problem
problem.solve()

# Output the optimal tour and its cost
optimal_path = []
for i in cities:
    for j in cities:
        if pulp.value(x[i][j]) == 1:
            optimal_path.append((i, j))

# Convert path indices to readable form and calculate total distance
readable_path = []
current_city = 0
total_distance = 0
for _ in range(n):
    next_city = next(j for i, j in optimal_path if i == current_city)
    total_distance += distances[current_city][next_city]
    readable_path.append(current_city)
    current_city = next_city
readable_path.append(readable_path[0])  # Return to starting city

# Convert city indices to names
readable_path = [city_names[city] for city in readable_path]

# Output
print("Optimal Path:", readable_path)
print("Total Distance:", total_distance)


Optimal Path: ['Abilene', 'Austin', 'San Antonio', 'Laredo', 'McAllen', 'Corpus Christi', 'Houston', 'Waco', 'Dallas', 'Wichita Falls', 'Amarillo', 'Lu bbock', 'Odessa', 'El Paso', 'Abilene']
Total Distance: 2549


In [2]:
import numpy as np
import pandas as pd
import pulp

# Read the distance matrix from the Excel file with headers and row names
df = pd.read_excel('TSP.xlsx', index_col=0)

# Extract the city names
city_names = df.columns.tolist()

# Convert the DataFrame to a NumPy array for the distance matrix
distances = df.to_numpy()

n = len(distances)  # Number of cities
cities = list(range(n))

# Create the problem variable to contain the problem data
problem = pulp.LpProblem("TSP", pulp.LpMinimize)

# Create variables: x[i, j] is 1 if the path goes from city i to city j, else 0
x = pulp.LpVariable.dicts("x", (cities, cities), cat='Binary', lowBound=0, upBound=1)

# Objective function: Minimize the sum of distances
problem += pulp.lpSum([distances[i][j] * x[i][j] for i in cities for j in cities if i != j])

# Each city must be arrived at from exactly one other city
for j in cities:
    problem += pulp.lpSum([x[i][j] for i in cities if i != j]) == 1

# Each city must be left by exactly one other city
for i in cities:
    problem += pulp.lpSum([x[i][j] for j in cities if i != j]) == 1

# Subtour elimination using Miller-Tucker-Zemlin (MTZ) constraints
u = pulp.LpVariable.dicts("u", cities, lowBound=0, upBound=n-1, cat='Continuous')
for i in cities:
    for j in cities:
        if i != j and i != 0 and j != 0:
            problem += u[i] - u[j] + n * x[i][j] <= n - 1

# Solve the problem
problem.solve()

# Output the optimal tour and its cost
optimal_path = []
for i in cities:
    for j in cities:
        if pulp.value(x[i][j]) == 1:
            optimal_path.append((i, j))

# Convert path indices to readable form and calculate total distance
readable_path = []
current_city = 0
total_distance = 0
for _ in range(n):
    next_city = next(j for i, j in optimal_path if i == current_city)
    total_distance += distances[current_city][next_city]
    readable_path.append(current_city)
    current_city = next_city
readable_path.append(readable_path[0])  # Return to starting city

# Convert city indices to names
readable_path = [city_names[city] for city in readable_path]

# Output
print("Optimal Path:", readable_path)
print("Total Distance:", total_distance)

# Verifying path uniqueness
unique_cities = set(readable_path[:-1])
if len(unique_cities) == n:
    print("Each city is visited exactly once.")
else:
    print("Error: Some cities are visited more than once or not at all.")


Optimal Path: ['Abilene', 'Austin', 'San Antonio', 'Laredo', 'McAllen', 'Corpus Christi', 'Houston', 'Waco', 'Dallas', 'Wichita Falls', 'Amarillo', 'Lu bbock', 'Odessa', 'El Paso', 'Abilene']
Total Distance: 2549
Each city is visited exactly once.
